# Structured output

OpenAI offers a functionality for defining a structure of the messages generated by LLMs, AutoGen enables this functionality by propagating `response_format` passed to your agents to the underlying client.

For more info on structured output, please check [here](https://platform.openai.com/docs/guides/structured-outputs)


````{=mdx}
:::info Requirements
Install `pyautogen`:
```bash
pip install pyautogen
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

## Set your API Endpoint

The [`config_list_from_json`](https://ag2ai.github.io/ag2/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4o", "gpt-4o-mini"],
    },
)

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

## Construct Agents

In [2]:
from pydantic import BaseModel


class ResponseModel(BaseModel):
    question: str
    short_answer: str
    reasoning: str
    difficulty: float


llm_config = {"config_list": config_list, "cache_seed": 42}

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
)

assistant = autogen.AssistantAgent(
    name="Assistant",
    llm_config=llm_config,
    response_format=ResponseModel,
)

## Start Chat

In [3]:
user_proxy.initiate_chat(assistant, message="What's up", max_turns=1, summary_method="last_msg")

User_proxy (to Assistant):

What's up

--------------------------------------------------------------------------------
[autogen.oai.client: 11-29 19:13:00] {921} INFO - Failed to cache response: Can't pickle <class 'openai.types.chat.parsed_chat_completion.ParsedChatCompletion[ResponseModel]'>: attribute lookup ParsedChatCompletion[ResponseModel] on openai.types.chat.parsed_chat_completion failed
Assistant (to User_proxy):

{"question":"What's the current date and time?","short_answer":"You can get the current date and time by executing the following code:","reasoning":"To get the current date and time, we can use Python's built-in datetime module which provides functions to get the current date and time. By executing the code snippet, you will be able to see the current date and time.","difficulty":1}

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': "What's up", 'role': 'assistant', 'name': 'User_proxy'}, {'content': '{"question":"What\'s the current date and time?","short_answer":"You can get the current date and time by executing the following code:","reasoning":"To get the current date and time, we can use Python\'s built-in datetime module which provides functions to get the current date and time. By executing the code snippet, you will be able to see the current date and time.","difficulty":1}', 'tool_calls': [], 'role': 'user', 'name': 'Assistant'}], summary='{"question":"What\'s the current date and time?","short_answer":"You can get the current date and time by executing the following code:","reasoning":"To get the current date and time, we can use Python\'s built-in datetime module which provides functions to get the current date and time. By executing the code snippet, you will be able to see the current date and time.","difficulty":1}', cost={'usage_including_cached_inf